## OCOM5203M Assignment 2 - Image Caption Generation [100 marks]

The maximum number of marks for each part are shown in the section headers. As indicated in the main heading above, the overall assessment carries a maximum of 100 marks.

This summative assessment is weighted 75% of the final grade for the module.

**Note:** This notebook includes two figures that are uploaded on the assessment page for viewing and downloading. Ensure that these figures are stored in the same directory as the notebook so they render properly when you load it.

### Motivation

Through this coursework, you will:

> 1. Understand the principles of text pre-processing and vocabulary building.
> 2. Gain experience working with an image to text model.
> 3. Use and compare two different text similarity metrics for evaluating an image to text model, and understand evaluation challenges.


### Setup and resources

Having a GPU will speed up the image feature extraction process. You can use Google Colab to complete this assingment.

Please implement the coursework using Python and PyTorch (i.e., Keras and TensorFlow are **not** accepted).

This coursework will use a subset of the [COCO "Common Objects in Context" dataset](https://cocodataset.org/) for image caption generation. COCO contains 330K images, of 80 object categories, and at least five textual reference captions per image. Our subset consists of 5029 of these images, each of which has five or more different descriptions of the salient entities and activities, and we will refer to it as COCO_5029.

To download the data:

> 1. **Images**: download the zip file "coco_subset_images.zip (812MB)" [here](https://leeds365-my.sharepoint.com/:f:/g/personal/busmnom_leeds_ac_uk/EuAH3b6a4g9IjTNhroLLXPoB6ho6cwxYSNh885ZzrktYZA?e=QGSYpf).
> 2. **Reference captions**: on the COCO [download page](https://cocodataset.org/#download), download the file named "2017 Train/Val annotations (241MB)".
> 3. **Image meta data**: as our set is a subset of full COCO dataset, we have created a CSV file containing relevant meta data for our particular subset of images. You can download it also from Drive, "coco_subset_meta.csv" at the same link as 1.


### Submission

Please submit the following:

> 1. Your completed Jupyter notebook file, in .ipynb format. Please do not submit the figures 
> 2. The .html version of your notebook. **Check that all cells have been run** and all outputs (including all graphs you would like to be marked) displayed in the .html for marking.
>    
**Note:** You can download the HTML version of your notebook in Jupyter Notebook via `File > Download as > HTML (.html)`. If the HTML option is not available in Google Colab, download your `.ipynb` file and either open it in Jupyter Notebook to export as HTML (`File > Download as > HTML (.html)`) or use the command `jupyter nbconvert --to html my-notebook.ipynb`.

Final note:

> **Please include in this notebook everything that you would like to be marked, including figures. Under each section, put the relevant code containing your solution. You may re-use functions you defined previously, but any new code must be in the relevant section.** Feel free to add as many code cells as you need under each section.

Enter your student ID in the cell below. Note that there may be a message on the Minerva submission page asking you to anonymise your submission. Please disregard that message and provide your ID below.

In [ ]:
# Student ID: <your student ID>

## Imports

Feel free to add to this section as needed.

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
import torchvision.models as models
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import os
import numpy as np

Detect which device (CPU/GPU) to use.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

The basic principle of our image-to-text model is as pictured in the diagram below, where an Encoder network encodes the input image as a feature vector by providing the outputs of the last fully-connected layer of a pre-trained CNN (we use [ResNet-152](https://arxiv.org/abs/1512.03385)). This pretrained network has been trained on the complete ImageNet dataset and is thus able to recognise common objects.

These features are then fed into a Decoder network along with the reference captions. As the image feature dimensions are large and sparse, the Decoder network includes a linear layer which downsizes them, followed by a batch normalisation layer to speed up training. Those resulting features, as well as the reference text captions, are passed into a recurrent network (we will use an RNN).

The reference captions used to compute loss are represented as numerical vectors via an embedding layer whose weights are learned during training.

![Encoder Decoder](encoder_decoder_diagramv2022.png)

The Encoder-Decoder network could be coupled and trained end-to-end, without saving features to disk; however, this requires iterating through the entire image training set during training. We can make the training more efficient by decoupling the networks.

We will first extract the feature representations of the images from the Encoder and save them on disk (Part 1). During training of the Decoder (Part 3), we only need to iterate over the image feature data and the reference captions.

### Organisation of the Notebook

> 1. Extracting image features [4 marks]
> 2. Text preparation of training and validation data [10 marks]
> 3. Training the decoder [28 marks]
> 4. Generating predictions on test data [8 marks]
> 5. Caption evaluation via BLEU score [10 marks]
> 6. Caption evaluation via Cosine similarity [17 makrs]
> 7. Comparing BLEU and Cosine similarity [20 marks]
> 8. Overall quality [3 marks]



## 1 Extracting image features [4 marks]

### 1.1 EncoderCNN

Read through the template EncoderCNN class below and complete the class.

In [ ]:
class EncoderCNN(nn.Module):
    def __init__(self):
        """Load a pretrained CNN and remove the top fully connected (fc) layer."""
        super(EncoderCNN, self).__init__()
        
        # TO COMPLETE
        # 1. Load the ResNet-152 pretrained model.
        # 2. Keep all layers except the final classification layer.

    def forward(self, images):
        """Extract feature vectors from input images."""
        
        # TO COMPLETE
        # 1. Disable gradients for the forward pass through the CNN.
        # 2. Pass the images through the modified network.
        # 3. Return the extracted features.


In [ ]:
# instantiate encoder and put into evaluation mode.
encoder = EncoderCNN().to(device)
encoder.eval()

### 1.2 Processing the images

Pass the images through the ```Encoder``` model, saving the resulting features for each image. You may like to use a ```Dataset``` and ```DataLoader``` to load the data in batches for faster processing, or you may choose to simply read in one image at a time from disk without any loaders.

Note that as this is a forward pass only, no gradients are needed. You will need to be able to match each image ID (the image name without file extension) with its features later, so we suggest either saving a dictionary of image ID: image features, or keeping a separate ordered list of image IDs.

Use the following provided ImageNet transform.

In [ ]:
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.Normalize((0.485, 0.456, 0.406),   # using ImageNet norms
                         (0.229, 0.224, 0.225))])

In [7]:
# TO COMPLETE

In [ ]:
# torch.save(features, 'features.pt')

## 2 Text preparation [10 marks]


### 2.1 Build the caption dataset

All our selected COCO_5029 images are from the official 2017 train set.

The ```coco_subset_meta.csv``` file includes the image filenames and unique IDs of all the images in our subset. The ```id``` column corresponds to each unique image ID.

The COCO dataset includes many different types of annotations: bounding boxes, keypoints, reference captions, and more. We are interested in the captioning labels. Open ```captions_train2017.json``` from the zip file downloaded from the COCO website. You are welcome to come up with your own way of doing it, but we recommend using the ```json``` package to initially inspect the data, then the ```pandas``` package to look at the annotations (if you read in the file as ```data```, then you can access the annotations dictionary as ```data['annotations']```).

Use ```coco_subset_meta.csv``` to cross-reference with the annotations from ```captions_train2017.json``` to get all the reference captions for each image in COCO_5029.

For example, you may end up with data looking like this (this is a ```pandas``` DataFrame, but it could also be several lists, or some other data structure/s):

<img src="df_caption_set.png" alt="images matched to caption" width="700"/>

In [8]:
# TO COMPLETE

### 2.2 Clean the captions

Create a cleaned version of each caption. If using dataframes, we suggest saving the cleaned captions in a new column; otherwise, if you are storing your data in some other way, create data structures as needed.

**A cleaned caption should be all lowercase, and consist of only alphabet characters.**

Print out 10 original captions next to their cleaned versions to facilitate marking.

In [9]:
# TO COMPLETE

### 2.3  Split the data

Split the data 70/10/20% into train/validation/test sets. **Be sure that each unique image (and all corresponding captions) only appear in a single set.**

Complete the provided function below, which takes a list of unique image IDs and a 3-way split ratio as input, and returns a split of the image IDs accordingly.

In [ ]:
def split_ids(image_id_list, train=0.7, valid=0.1, test=0.2):
    """
    Args:
        image_id_list (list): List of unique image IDs.
        train (float): Proportion of the dataset to allocate for training.
        valid (float): Proportion of the dataset to allocate for validation.
        test (float): Proportion of the dataset to allocate for testing.
    Returns:
        tuple: Three lists containing IDs for the training, validation, and test sets.
    """
    # TO COMPLETE 
    # 1. Calculate the number of items in each split (train, valid, test).
    # 2. Use list slicing to create the splits.
    # Think: How might you ensure the sets are representative of the overall dataset?

    return train_ids, valid_ids, test_ids


### 2.4 Building the vocabulary

The `Vocabulary` class provided below serves as a simple wrapper to map unique words to integer IDs, facilitating word-to-integer and integer-to-word conversions.

1. **Complete the `add_word` Method:** Implement the logic to add a new word to the vocabulary. Ensure that each word is assigned a unique integer ID, starting from `3` (after the special tokens).

2. **Complete the `__call__` Method:** Implement the logic to retrieve the integer ID for a given word. If the word is not found in the vocabulary, return the ID for the `<unk>` token.

Refer to the class template below and complete the indicated sections.


In [ ]:
class Vocabulary(object):
    """ Simple vocabulary wrapper which maps every unique word to an integer ID. """
    def __init__(self):
        # intially, set both the IDs and words to dictionaries with special tokens
        self.word2idx = {'<pad>': 0, '<unk>': 1, '<end>': 2}
        self.idx2word = {0: '<pad>', 1: '<unk>', 2: '<end>'}
        self.idx = 3

    # TO COMPLETE
    # Implement the logic to add a word to the vocabulary
    def add_word(self, word):
        pass

    # TO COMPLETE
    # Implement logic to retrieve the integer ID for a word
    # Return <unk>'s ID if the word is not found
    def __call__(self, word):
        pass

    def __len__(self):
        return len(self.word2idx)

Collect all words from the cleaned captions in the **training and validation sets**, ignoring any words which appear 3 times or less; this should leave you with roughly 2200 words (plus or minus is fine). As the vocabulary size affects the embedding layer dimensions, it is better not to add the very infrequently used words to the vocabulary.

Create an instance of the ```Vocabulary()``` object and add all your words to it.

In [ ]:
# TO COMPLETE

### 2.5 The Dataset and DataLoader

Create a PyTorch ```Dataset``` class and a corresponding ```DataLoader``` for the inputs to the decoder. Create three sets: one each for training, validation, and test. Set ```shuffle=True``` for the training set DataLoader.

The ```Dataset``` function ```__getitem__(self, index)``` should return three Tensors:

>1. A Tensor of image features, dimension (1, 2048).
>2. A Tensor of integer word ids representing the reference caption; use your ```Vocabulary``` object to convert each word in the caption to a word ID. Be sure to add the word ID for the ```<end>``` token at the end of each caption, then fill in the the rest of the caption with the ```<pad>``` token so that each caption has uniform lenth (max sequence length) of **47**.
>3. A Tensor of integers representing the true lengths of every caption in the batch (include the ```<end>``` token in the count).


Note that as each unique image has five or more (say, ```n```) reference captions, each image feature will appear ```n``` times, once in each unique (feature, caption) pair.

In [ ]:
class COCO_Subset(Dataset):
    """ COCO subset custom dataset, compatible with torch.utils.data.DataLoader. """

    def __init__(self, df, features, vocab):
        """
        Args:
            df: (dataframe or some other data structure/s you may prefer to use)
            features: image features
            vocab: vocabulary wrapper

        """

        # TO COMPLETE


    def __getitem__(self, index):
        """ Returns one data tuple (feature [1, 2048], target caption of word IDs [1, 47], and integer true caption length) """

       # TO COMPLETE


    def __len__(self):
        return len(self.features)


Load one batch of the training set and print out the shape of each returned Tensor.

In [ ]:
# train_iter = iter(train_loader)
# features, captions, lengths = train_iter.next()
# print(features.shape)
# print(captions.shape)
# print(lengths.shape)

## 3 Train DecoderRNN [28 marks]

### 3.1 Define the decoder model

In this section, you will complete the implementation of the `DecoderRNN` class by defining the missing components in the `__init__` and `forward` methods.

1. **Initialize the Decoder:**
   - Add the required layers such as a linear transformation, batch normalization, an embedding layer, and an RNN.

2. **Forward Pass:**
   - Implement the logic for processing image features and tokenized captions through the decoder. 
   - Ensure proper sequence handling with `pack_padded_sequence()`.

3. **Answer Questions:**
   - In a separate cell given below, answer all the questions embedded in the  ode comments. Use m. explanations.

Refer to the provided comments and [PyTorch documentation](https://pytorch.org/docs/stable/index.html) as needed. The code comments outline the functionality for each step, guiding your implementation.mplementation.


In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, vocab_size, embed_size=256, hidden_size=512, num_layers=1, max_seq_length=47):
        """
        Initialize the DecoderRNN.
        Inputs:
        - vocab_size: Size of the vocabulary (number of unique words in captions)
        - embed_size: Dimensionality of word embeddings (size of W_emb in the schematic)
        - hidden_size: Size of the RNN's hidden state
        - num_layers: Number of layers in the RNN (you may or may not use this)
        - max_seq_length: Maximum length of the generated captions
        """
        super(DecoderRNN, self).__init__()

        # RECALL: Revisit the schematic of the encoder/decoder architecture given above and in the assessment page.
        # The input to the DecoderRNN is the image feature vector from the EncoderCNN (size: 2048).
        # The Decoder first transforms this vector using a linear layer followed by a batch normalization layer.

        # THINK: What should be the input and output dimensions of this layer? Why?

        # QUESTION 1: Why do we need to transform the 2048-dimensional image feature vector before passing it to the RNN?

        # QUESTION 2: Why might batch normalization be useful after resizing the image 
        # feature vector? What potential issues could arise if batch normalization is omitted?

        # Define the Linear transformation (Linear + BN)
        # TO COMPLETE <your code>

        # RECALL: The forward pass of the decoder RNN requires an embedding representation of 
        # the input tokens/words.

        # Read about the `torch.nn.Embedding` class and implement this embedding layer.
        # TO COMPLETE <your code>

        # QUESTION 3: What is the purpose of the embedding layer in the DecoderRNN? 
        # Could you use a pretrained word embedding instead of training it from scratch? 
        # If so, what are the potential advantages and disadvantages of using a pretrained embedding?

        # Define the RNN architecture here using hidden units you deem appropriate 
        # You may experiment with different hidden unit types and number of layers.
        self.rnn = # TO COMPLETE <your code>

        # QUESTION 4:
        # Overfitting and underfitting are common issues when training machine learning models.
        # 
        # Answer the following:
        #
        # Q4.1. What architectural choices in your RNN design (e.g., number of hidden units, layers, or type of RNN) 
        # could contribute to overfitting/underfitting? Explain your reasoning.
        #
        # Q4.2. How would you evaluate whether your model is overfitting or underfitting the data? What metrics or observations 
        # would you look for during training?
        #
        # Q4.3. What strategies or techniques would you use to guard against overfitting and underfitting in your model? 

        # RECALL: The RNN processes sequences (image features + word embeddings) 
        # and produces hidden states that represent the context of the caption.

        # Finally, the output of the RNN hidden state needs to be mapped and converted 
        # to probabilities over the vocabulary.
        # TO COMPLETE <your code>

        # Maximum sequence length
        self.max_seq_length = max_seq_length

    
    def forward(self, features, captions, lengths):
        """
        Decode image feature vectors and generate captions.
        Inputs:
        - features: Image features extracted from the EncoderCNN.
        - captions: Padded tokenized captions.
        - lengths: Actual lengths of captions before padding. You will need this for the pack_padded_sequence function below.
        Outputs:
        - outputs: Vocabulary predictions for each time step.
        """
    
        # STEP 1: Transform the image features using the Linear + BN layer you defined earlier
        # Use the transformations you implemented in the __init__ function.
        im_features = # TO COMPLETE <your code>
    
        # STEP 2: Convert captions to embeddings using the embedding layer you defined above
        # Use the embedding layer you defined earlier to process `captions`.
        embeddings = # TO COMPLETE <your code>
    
        # STEP 3: Concatenate image features and word embeddings
        # Use torch.cat to concatenate `im_features` with `embeddings`
        # Hint: You may need to use the unsqueeze() function on the im_features
        embeddings = # TO COMPLETE <your code>
    
        # NOTE: PyTorch's `pack_padded_sequence`:
        # This function removes padding tokens and ensures efficient processing of variable-length sequences by the RNN.
        # You must use the `lengths` input to create the `packed` sequence.
    
        # STEP 4: Consult PyTorch's documentation to learn about how to use `pack_padded_sequence`
        # to create a variable called `packed` that the RNN will process in the next step.
        packed = # TO COMPLETE <your code>
    
        # STEP 5: Processing the packed sequences with the RNN
        hiddens, _ = self.rnn(packed)
    
        # STEP 6: Transform the hidden states to output vocabulary probabilities
        # Use the Linear transformation(s) you defined earlier to map hidden states to vocabulary size.
        outputs = # TO COMPLETE <your code>
    
        return outputs



In [ ]:
# instantiate decoder
# You may experiment with different values for the embedding size, hidden size, etc (checked the default values in function signature above).
decoder = DecoderRNN(len(vocab)).to(device)

#### Answers to Questions 1-4
Use this cell to answer questions 1-4 in the `DecoderRNN`.

### 3.2 Train the decoder

Train the decoder by passing the features, reference captions, and targets to the decoder, then computing loss based on the outputs and the targets. Note that when passing the targets and model outputs to the loss function, the targets will also need to be formatted using ```pack_padded_sequence()```.

We recommend a batch size of around 64 (though feel free to adjust as necessary for your hardware).

We recommend saving a checkpoint of your trained model after training so you don't need to re-train multiple times.

Display a graph of training and validation loss over epochs to justify your stopping point. Additionally, document the different RNN units and design choices you experimented with, including the rationale behind each choice. Describe the configurations tested, the observations made during training and validation, and how these observations influenced subsequent changes to your model design.

In [ ]:
# TO COMPLETE

## 4 Generate predictions on test data [8 marks]

Display 5 sample test images containing different objects, along with your model’s generated captions and all the reference captions for each.

> Remember that everything **displayed** in the submitted notebook and .html file will be marked, so be sure to run all relevant cells.

In [ ]:
# TO COMPLETE

## 5 Caption evaluation using BLEU score [10 marks]

There are different methods for measuring the performance of image to text models. We will evaluate our model by measuring the text similarity between the generated caption and the reference captions, using two commonly used methods. Ther first method is known as *Bilingual Evaluation Understudy (BLEU)*.

###  5.1 BLEU score


One common way of comparing a generated text to a reference text is using BLEU. This article gives a good intuition to how the BLEU score is computed: https://machinelearningmastery.com/calculate-bleu-score-for-text-python/, and you may find an implementation online to use. One option is the NLTK implementation `nltk.translate.bleu_score` here: https://www.nltk.org/api/nltk.translate.bleu_score.html


> **Tip:** BLEU scores can be weighted by ith-gram. Check that your scores make sense; and feel free to use a weighting that best matches the data. We will not be looking for specific score ranges; rather we will check that the scores are reasonable and meaningful given the captions.

Write the code to evaluate the trained model on the complete test set and calculate the BLEU score using the predictions, compared against all five references captions.

Display a histogram of the distribution of scores over the test set.

In [ ]:
# TO COMPLETE

### 5.2 BLEU score examples

Find one sample with high BLEU score and one with a low score, and display the model's predicted sentences, the BLEU scores, and the 5 reference captions.

In [ ]:
# TO COMPLETE

## 6 Caption evaluation using cosine similarity [17 marks]

###  6.1 Cosine similarity

The cosine similarity measures the cosine of the angle between two vectors in n-dimensional space. The smaller the angle, the greater the similarity.

To use the cosine similarity to measure the similarity between the generated caption and the reference captions:

* Find the embedding vector of each word in the caption
* Compute the average vector for each caption
* Compute the cosine similarity score between the average vector of the generated caption and average vector of each reference caption
* Compute the average of these scores

Calculate the cosine similarity using the model's predictions over the whole test set.

Display a histogram of the distribution of scores over the test set.

In [ ]:
# TO COMPLETE

#### 6.2 Cosine similarity examples

Find one sample with high cosine similarity score and one with a low score, and display the model's predicted sentences, the cosine similarity scores, and the 5 reference captions.

In [ ]:
# TO COMPLETE

## 7 Comparing BLEU and Cosine similarity [20 marks]

### 7.1 Test set distribution of scores

Compare the model’s performance on the test set evaluated using BLEU and cosine similarity and discuss some weaknesses and strengths of each method (explain in words, in a text box below).

Please note, to compare the average test scores, you need to rescale the Cosine similarity scores [-1 to 1] to match the range of BLEU method [0.0 - 1.0].

In [ ]:
# TO COMPLETE

 ### 7.2 Analysis of individual examples

Find and display one example where both methods give similar scores and another example where they do not and discuss. Include both scores, predicted captions, and reference captions.

In [ ]:
# TO COMPLETE

### 8. Overall quality [3 marks]

- **Code Modularity:** The implementation should follow a clean, modular design, with well-structured functions and classes (where approperiate).
- **Readability:** Code should be clear with meaningful variable names and appropriate use of comments.
- **Output Accessibility:** Outputs should be neatly formatted and easy to interpret, avoiding excessive or unnecessary information dumping into the output.
- **Clarity in Visualisation:** Graphs and visualisations results should be well-labelled.